In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings 
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim

In [ ]:
'''In the code cell above, Nominatim refers to the geocoding software that will be used to generate locations.

We begin by instantiating the geocoder. Then, we need only apply the name or address as a Python string. (In this case, we supply "Pyramid of Khufu", also known as the Great Pyramid of Giza.)

If the geocoding is successful, it returns a geopy.location.Location object with two important attributes:

the "point" attribute contains the (latitude, longitude) location, and
the "address" attribute contains the full address.'''

In [ ]:
geolocator = Nominatim(user_agent="kaggle_learn")
location = geolocator.geocode("Pyramid of Khufu")

print(location.point)
print(location.address)

In [ ]:
# The value for the "point" attribute is a geopy.point.Point object, and we can get the latitude and longitude from the latitude and longitude attributes, respectively.

point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

In [ ]:
# It's often the case that we'll need to geocode many different addresses. For instance, say we want to obtain the locations of 100 top universities in Europe.

universities = pd.read_csv("../input/geospatial-learn-course-data/top_universities.csv")
universities.head()

In [ ]:
# Then we can use a lambda function to apply the geocoder to every row in the DataFrame.

def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}
universities.head()

In [ ]:
# visualize all of the locations that were returned by the geocoder. 

# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
m

In [ ]:
# Table joins
# Combine data from different sources.

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]
europe_boundaries.head()

In [ ]:
europe_stats.head()

In [ ]:
# We do the attribute join in the code cell below. The on argument is set to the column name that is used to match rows in europe_boundaries to rows in europe_stats.

# Use an attribute join to merge data about countries in Europe
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

In [ ]:
# Then we can use a spatial join to match each university to its corresponding country. We do this with gpd.sjoin().

# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)

# Investigate the result
print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))

european_universities.head()

In [ ]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import folium 
from folium import Marker
from folium.plugins import MarkerCluster

In [ ]:
# Using the embed_map() function to visualize your maps.
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [ ]:
# 1) Geocode the missing locations.
# create a DataFrame starbucks containing Starbucks locations in the state of California.

# Load and preview Starbucks locations in California
starbucks = pd.read_csv("../input/geospatial-learn-course-data/starbucks_locations.csv")
starbucks.head()

In [ ]:
# Most of the stores have known (latitude, longitude) locations. But, all of the locations in the city of Berkeley are missing.

# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

In [ ]:
'''Use the code cell below to fill in these values with the Nominatim geocoder.

Note that in the tutorial, we used Nominatim() (from geopy.geocoders) to geocode values, and this is what you can use in your own projects outside of this course.

In this exercise, you will use a slightly different function Nominatim() (from learntools.geospatial.tools). This function was imported at the top of the notebook and works identically to the function from GeoPandas.

So, in other words, as long as:

you don't change the import statements at the top of the notebook, and
you call the geocoding function as geocode() in the code cell below,
your code will work as intended!'''

In [ ]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

In [ ]:
# 2) View Berkeley locations.Visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style.

# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Show the map
embed_map(m_2, 'q_2.html')

In [ ]:
# 3) Consolidate your data.

CA_counties = gpd.read_file("../input/geospatial-learn-course-data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.crs = {'init': 'epsg:4326'}
CA_counties.head()

In [ ]:
'''Next, we create three DataFrames:

CA_pop contains an estimate of the population of each county.
CA_high_earners contains the number of households with an income of at least $150,000 per year.
CA_median_age contains the median age for each county.'''

In [ ]:
CA_pop = pd.read_csv("../input/geospatial-learn-course-data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("../input/geospatial-learn-course-data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("../input/geospatial-learn-course-data/CA_county_median_age.csv", index_col="GEOID")

In [ ]:
# join the CA_counties GeoDataFrame with CA_pop, CA_high_earners, and CA_median_age.
# Name the resultant GeoDataFrame CA_stats, and make sure it has 8 columns: "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age".

cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")

In [ ]:
# Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns. Create a "density" column with the population density.

CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

In [ ]:
'''4) Which counties look promising?
Collapsing all of the information into a single GeoDataFrame also makes it much easier to select counties that meet specific criteria.

Use the next code cell to create a GeoDataFrame sel_counties that contains a subset of the rows (and all of the columns) from the CA_stats GeoDataFrame. In particular, you should select counties where:

there are at least 100,000 households making $150,000 per year,
the median age is less than 38.5, and
the density of inhabitants is at least 285 (per square kilometer).
Additionally, selected counties should satisfy at least one of the following criteria:

there are at least 500,000 households making $150,000 per year,
the median age is less than 35.5, or
the density of inhabitants is at least 1400 (per square kilometer).'''

In [ ]:
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]

In [ ]:
# 5) How many stores did you identify?
# create a GeoDataFrame starbucks_gdf with all of the starbucks locations.

starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)

In [ ]:
# 6) Visualize the store locations.
# Create a map that shows the locations of the stores that you identified in the previous question.

# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)

# Show the map
embed_map(m_6, 'q_6.html')

In [ ]:
import folium
from folium import Marker, GeoJson
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd

In [ ]:
# a dataset from the US Environmental Protection Agency (EPA) that tracks releases of toxic chemicals in Philadelphia, Pennsylvania, USA.

releases = gpd.read_file("../input/geospatial-learn-course-data/toxic_release_pennsylvania/toxic_release_pennsylvania/toxic_release_pennsylvania.shp") 
releases.head()

In [ ]:
# a dataset that contains readings from air quality monitoring stations in the same city.

stations = gpd.read_file("../input/geospatial-learn-course-data/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations.shp")
stations.head()

In [ ]:
'''Measuring distance
To measure distances between points from two different GeoDataFrames, we first have to make sure that they use the same coordinate reference system (CRS). Thankfully, this is the case here, where both use EPSG 2272.

'''

In [ ]:
print(stations.crs)
print(releases.crs)

In [ ]:
# calculates the distance (in feet) between a relatively recent release incident in recent_release and every station in the stations GeoDataFrame.

# Select one release incident in particular
recent_release = releases.iloc[360]

# Measure distance from release to each station
distances = stations.geometry.distance(recent_release.geometry)
distances

In [ ]:
# Using the calculated distances, we can obtain statistics like the mean distance to each station.

print('Mean distance to monitoring stations: {} feet'.format(distances.mean()))

In [ ]:
# Or, we can get the closest monitoring station

print('Closest monitoring station ({} feet):'.format(distances.min()))
print(stations.iloc[distances.idxmin()][["ADDRESS", "LATITUDE", "LONGITUDE"]])

In [ ]:
# Creating a buffer
# If we want to understand all points on a map that are some radius away from a point, the simplest way is to create a buffer.
# The code cell below creates a GeoSeries two_mile_buffer containing 12 different Polygon objects. Each polygon is a buffer of 2 miles (or, 2*5280 feet) around a different air monitoring station.

two_mile_buffer = stations.geometry.buffer(2*5280)
two_mile_buffer.head()

In [ ]:
# We use folium.GeoJson() to plot each polygon on a map. Note that since folium requires coordinates in latitude and longitude, we have to convert the CRS to EPSG 4326 before plotting.

# Create map with release incidents and monitoring stations
m = folium.Map(location=[39.9526,-75.1652], zoom_start=11)
HeatMap(data=releases[['LATITUDE', 'LONGITUDE']], radius=15).add_to(m)
for idx, row in stations.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(m)
    
# Plot each polygon on the map
GeoJson(two_mile_buffer.to_crs(epsg=4326)).add_to(m)

# Show the map
m

In [ ]:
'''Now, to test if a toxic release occurred within 2 miles of any monitoring station, we could run 12 different tests for each polygon (to check individually if it contains the point).

But a more efficient way is to first collapse all of the polygons into a MultiPolygon object. We do this with the unary_union attribute.'''

In [ ]:
# Turn group of polygons into single multipolygon
my_union = two_mile_buffer.geometry.unary_union
print('Type:', type(my_union))

# Show the MultiPolygon object
my_union

In [ ]:
'''We use the contains() method to check if the multipolygon contains a point. We'll use the release incident from earlier in the tutorial, which we know is roughly 3781 feet to the closest monitoring station.'''

In [ ]:
# The closest station is less than two miles away
my_union.contains(releases.iloc[360].geometry)

In [ ]:
# But not all releases occured within two miles of an air monitoring station!

# The closest station is more than two miles away
my_union.contains(releases.iloc[358].geometry)